<a href="https://colab.research.google.com/github/Saifullah785/Deep_Learning_using_PyTorch/blob/main/Lecture_10_ANN_Fashion_Mnist_Pytorch_Gpu_Optimized_Optuna/Lecture_10_ANN_Fashion_Mnist_Pytorch_Gpu_Optimized_Optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
torch.manual_seed(42)